In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV

In [ ]:
train = pd.read_csv('/content/Training.csv')
test  = pd.read_csv('/content/Testing.csv')

print("Raw train shape:", train.shape)
print("Raw test shape: ", test.shape)
display(train.head())
display(test.head())

Raw train shape: (4920, 134)
Raw test shape:  (42, 133)


,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis,Unnamed: 133
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN


,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV

### Load Data

In [ ]:
train = pd.read_csv('/content/Training.csv')
test  = pd.read_csv('/content/Testing.csv')

print("Raw train shape:", train.shape)
print("Raw test shape: ", test.shape)
display(train.head())
display(test.head())

NameError: name 'pd' is not defined

### Remove Unnamed Columns

In [ ]:
# Remove unnamed columns
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]
test  = test.loc[:, ~test.columns.str.contains('^Unnamed')]

print("Train shape after dropping unnamed:", train.shape)
print("Test shape after dropping unnamed: ", test.shape)

NameError: name 'train' is not defined

### Drop Duplicates

In [ ]:
print("Train duplicates before:", train.duplicated().sum())
train = train.drop_duplicates()
print("Train shape after dropping duplicates:", train.shape)

NameError: name 'train' is not defined

### Align Test Set Features

In [ ]:
# Ensure that the test has same feature columns as train (excluding target)
target = 'prognosis'
train_feats = [c for c in train.columns if c != target]
test_feats  = [c for c in test.columns if c != target]

# Add any missing train feature columns to test (fill with 0)
for c in train_feats:
    if c not in test_feats:
        test[c] = 0

# Reorder test columns to match train
test = test[[*train_feats, target]]

print("Aligned test shape:", test.shape)

NameError: name 'train' is not defined

### Remove Overlapping Rows

In [ ]:
# Create hashes for each full row (features + target) to detect exact duplicates across files
train['_row_hash'] = pd.util.hash_pandas_object(train, index=False)
test['_row_hash']  = pd.util.hash_pandas_object(test, index=False)

common_hashes = set(train['_row_hash']).intersection(set(test['_row_hash']))
print("Exact train/test overlapping rows:", len(common_hashes))

# Drop overlapping rows from train
train = train[~train['_row_hash'].isin(common_hashes)].drop(columns=['_row_hash'])
test  = test.drop(columns=['_row_hash'])
print("Train shape after removing overlaps:", train.shape)
print("Test shape after cleaning:", test.shape)

NameError: name 'pd' is not defined

### Prepare Data for Modeling

In [ ]:
X = train.drop(columns=[target]).astype(int)
y_raw = train[target].astype(str)

X_test = test.drop(columns=[target]).astype(int)
y_test_raw = test[target].astype(str)

le = LabelEncoder()
y = le.fit_transform(y_raw)
y_test = le.transform(y_test_raw)  # will fail if test has unseen labels — check earlier

print("Classes (label encoder):", list(le.classes_))
print("X shape:", X.shape, "y shape:", y.shape)

NameError: name 'train' is not defined

### Check Data Distribution

In [ ]:
print("Unique rows in X (post-clean):", X.duplicated().sum(), "duplicates remain")
print("Train class distribution (counts):")
display(pd.Series(y).value_counts().sort_index())
print("Test class distribution (counts):")
display(pd.Series(y_test).value_counts().sort_index())

NameError: name 'X' is not defined

### Split Data

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Internal split: train", X_tr.shape, "val", X_val.shape)

NameError: name 'train_test_split' is not defined

### Save Cleaned Data

In [ ]:
train.to_csv('/content/Training_Cleaned.csv', index=False)
test.to_csv('/content/Testing_Cleaned.csv', index=False)

print("Cleaned datasets saved successfully!")

NameError: name 'train' is not defined

### Download Cleaned Data

In [ ]:
from google.colab import files

files.download('/content/Training_Cleaned.csv')
files.download('/content/Testing_Cleaned.csv')

FileNotFoundError: Cannot find file: /content/Training_Cleaned.csv

In [ ]:
# Remove unnamed columns
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]
test  = test.loc[:, ~test.columns.str.contains('^Unnamed')]

print("Train shape after dropping unnamed:", train.shape)
print("Test shape after dropping unnamed: ", test.shape)

Train shape after dropping unnamed: (4920, 133)
Test shape after dropping unnamed:  (42, 133)


In [ ]:
print("Train duplicates before:", train.duplicated().sum())
train = train.drop_duplicates()
print("Train shape after dropping duplicates:", train.shape)

Train duplicates before: 4616
Train shape after dropping duplicates: (304, 133)


In [ ]:
# Ensure that the test has same feature columns as train (excluding target)
target = 'prognosis'
train_feats = [c for c in train.columns if c != target]
test_feats  = [c for c in test.columns if c != target]

# Add any missing train feature columns to test (fill with 0)
for c in train_feats:
    if c not in test_feats:
        test[c] = 0

# Reorder test columns to match train
test = test[[*train_feats, target]]

print("Aligned test shape:", test.shape)

Aligned test shape: (42, 133)


In [ ]:
# Create hashes for each full row (features + target) to detect exact duplicates across files
train['_row_hash'] = pd.util.hash_pandas_object(train, index=False)
test['_row_hash']  = pd.util.hash_pandas_object(test, index=False)

common_hashes = set(train['_row_hash']).intersection(set(test['_row_hash']))
print("Exact train/test overlapping rows:", len(common_hashes))

# Drop overlapping rows from train
train = train[~train['_row_hash'].isin(common_hashes)].drop(columns=['_row_hash'])
test  = test.drop(columns=['_row_hash'])
print("Train shape after removing overlaps:", train.shape)
print("Test shape after cleaning:", test.shape)

Exact train/test overlapping rows: 41
Train shape after removing overlaps: (263, 133)
Test shape after cleaning: (42, 133)


In [ ]:
X = train.drop(columns=[target]).astype(int)
y_raw = train[target].astype(str)

X_test = test.drop(columns=[target]).astype(int)
y_test_raw = test[target].astype(str)

le = LabelEncoder()
y = le.fit_transform(y_raw)
y_test = le.transform(y_test_raw)  # will fail if test has unseen labels — check earlier

print("Classes (label encoder):", list(le.classes_))
print("X shape:", X.shape, "y shape:", y.shape)

Classes (label encoder): ['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne', 'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma', 'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis', 'Common Cold', 'Dengue', 'Diabetes ', 'Dimorphic hemmorhoids(piles)', 'Drug Reaction', 'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack', 'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E', 'Hypertension ', 'Hyperthyroidism', 'Hypoglycemia', 'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine', 'Osteoarthristis', 'Paralysis (brain hemorrhage)', 'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis', 'Typhoid', 'Urinary tract infection', 'Varicose veins', 'hepatitis A']
X shape: (263, 132) y shape: (263,)


In [ ]:
print("Unique rows in X (post-clean):", X.duplicated().sum(), "duplicates remain")
print("Train class distribution (counts):")
display(pd.Series(y).value_counts().sort_index())
print("Test class distribution (counts):")
display(pd.Series(y_test).value_counts().sort_index())


Unique rows in X (post-clean): 0 duplicates remain
Train class distribution (counts):


,count
0,6
1,4
2,4
3,7
4,4
5,5
6,6
7,5
8,9
9,7


Test class distribution (counts):


,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Internal split: train", X_tr.shape, "val", X_val.shape)

Internal split: train (210, 132) val (53, 132)


In [ ]:
train.to_csv('/content/Training_Cleaned.csv', index=False)
test.to_csv('/content/Testing_Cleaned.csv', index=False)

print("Cleaned datasets saved successfully!")

Cleaned datasets saved successfully!


In [ ]:
from google.colab import files

files.download('/content/Training_Cleaned.csv')
files.download('/content/Testing_Cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>